In [1]:
import angr
import claripy
from angr.sim_type import SimTypePointer, SimTypeFunction, SimTypeChar, SimTypeInt, parse_defns
from angr.errors import AngrCallableMultistateError

In [2]:
#angr logging is way too verbose
import logging
log_things = ["angr", "pyvex", "claripy", "cle"]
for log in log_things:
    logger = logging.getLogger(log)
    logger.disabled = True
    logger.propagate = False

In [3]:
password = claripy.BVS('pass_arg', 9*8)
pass_arg = angr.PointerWrapper(password, buffer=True)
simfile = angr.SimFile('userargs', content='password')
charstar = SimTypePointer(SimTypeChar())
prototype = SimTypeFunction((charstar, charstar), SimTypeInt(False))

In [4]:
def loadproj(p, addr):
    state = p.factory.call_state(addr, "userargs", pass_arg, prototype=prototype)
    state.fs.insert("userargs", simfile)
    simgr = p.factory.simgr(state)
    simgr.run()
    return simgr

In [5]:
def getconcrete(p, addr, password):
    state = p.factory.call_state(addr, "userargs", password, prototype=prototype)
    state.fs.insert("userargs", simfile)
    simgr = p.factory.simgr(state)
    simgr.run()
    return simgr.deadended[0].regs.eax._model_concrete.value

In [6]:
#fauxware analysis

In [7]:
p = angr.Project('fauxware', auto_load_libs=False)
simgr = loadproj(p, 0x401209)
auth_args = []
for end in simgr.deadended:
    auth_args.append([end.solver.eval(password,cast_to=bytes),
                     end.regs.eax._model_concrete.value])
for arg in auth_args:
    print(arg)

The main binary is a position-independent executable. It is being loaded with a base address of 0x400000.


[b'SOSNEAKY\x00', 1]
[b'\x01\x01\x01\x01\x01\x01\x01\x00\x00', 0]
[b'password\x00', 1]


In [8]:
def auth(thing):
    return getconcrete(p, 0x401209, thing)

In [9]:
#pmod fauxware with functionally identical strcmp

In [10]:
p_mod = angr.Project('fauxware-mod', auto_load_libs=False)
simgr_mod = loadproj(p_mod, 0x4011e9)
auth_mod_args = []
for end in simgr_mod.deadended:
    auth_mod_args.append([end.solver.eval(password,cast_to=bytes),
                     end.regs.eax._model_concrete.value])
for arg in auth_mod_args:
    print(arg)

The main binary is a position-independent executable. It is being loaded with a base address of 0x400000.


[b'\xac\x00\x00\x00\x00\x00\x00\x00\x00', 0]
[b'S\x00\x00\x00\x00\x00\x00\x00\x00', 0]
[b'p\x00\x00\x00\x00\x00\x00\x00\x00', 0]
[b'SO\x00\x00\x00\x00\x00\x00\x00', 0]
[b'pa\x00\x00\x00\x00\x00\x00\x00', 0]
[b'SOS\x00\x00\x00\x00\x00\x00', 0]
[b'pas\x00\x00\x00\x00\x00\x00', 0]
[b'SOSN\x00\x00\x00\x00\x00', 0]
[b'pass\x00\x00\x00\x00\x00', 0]
[b'SOSNEAKY\x00', 1]
[b'SOSNE\x00\x00\x00\x00', 0]
[b'passw\x00\x00\x00\x00', 0]
[b'SOSNEA\x00\x00\x00', 0]
[b'passwo\x00\x00\x00', 0]
[b'SOSNEAK\x00\x00', 0]
[b'passwor\x00\x00', 0]
[b'SOSNEAKY\x01', 0]
[b'password\x01', 0]
[b'password\x00', 1]


In [11]:
def auth_mod(password):
    return getconcrete(p_mod, 0x4011e9, password)

In [12]:
#different authentication than fauxware

In [13]:
p_diff = angr.Project('fauxware-diff', auto_load_libs=False)
simgr_diff = loadproj(p_diff, 0x401209)
auth_diff_args = []
for end in simgr_diff.deadended:
    auth_diff_args.append([end.solver.eval(password,cast_to=bytes),
                     end.regs.eax._model_concrete.value])
for arg in auth_diff_args:
    print(arg)

The main binary is a position-independent executable. It is being loaded with a base address of 0x400000.


[b'\x00\x00\x00\x00\x00\x00\x00\x00\x00', 0]
[b'\x01\x00\x00\x00\x00\x00\x00\x00\x00', 0]
[b'\x01\xff\x00\x00\x00\x00\x00\x00\x00', 0]
[b'\xff\xff\x80\x00\x01\x04\x08\x08\x10', 0]
[b'RNRMD@JX\x00', 1]
[b'\x01\x01\x01\x01\x00\x01\x02\x04\x80', 0]
[b'RNRMD@JX\x80', 1]
[b'\x01\x01\x01\x01\xff\x00\x00\x00\x00', 0]
[b'\x10\x01\x02\x01\x80\x01\x00\x04\x80', 0]
[b'\x80\x10\x01\x10 \x02\xff\x00\x00', 0]
[b' \x04\x02\x10\x10\x01\x02\x04\x00', 0]
[b'password\x00', 1]
[b'\xff\xff\xff\xff\xff\xff@\xff\x04', 0]


In [14]:
def auth_diff(password):
    return getconcrete(p_diff, 0x401209, password)

In [15]:
#Analysis of equivalence of fauxware and itself

In [16]:
orig_orig_diff = []
for case in auth_args:
    if auth(case[0]) != case[1]:
        print("pass = " + str(case[0]) + ", result = " + str(auth(case[0])) +" actual = "+str(case[1]))
        orig_orig_diff.append(case[0])
print()
for case in auth_args:
    if auth(case[0]) != case[1]:
        print("pass = " + str(case[0]) + ", result = " + str(auth(case[0])) +" actual = "+str(case[1]))
        orig_orig_diff.append(case[0])
print(orig_orig_diff)


[]


In [17]:
#Analysis between fauxware (auth_args) and fauxware-mod (auth_mod_args) should be functionally equiv

In [18]:
orig_mod_diff = []
for case in auth_args:
    if auth_mod(case[0]) != case[1]:
        print("pass = " + str(case[0]) + ", result = " + str(auth_mod(case[0])) +" actual = "+str(case[1]))
        orig_mod_diff.append(case[0])
print()
for case in auth_mod_args:
    if auth(case[0]) != case[1]:
        print("pass = " + str(case[0]) + ", result = " + str(auth(case[0])) +" actual = "+str(case[1]))
        orig_mod_diff.append(case[0])
print(orig_mod_diff)


pass = b'S\x00\x00\x00\x00\x00\x00\x00\x00', result = 1 actual = 0
pass = b'p\x00\x00\x00\x00\x00\x00\x00\x00', result = 1 actual = 0
pass = b'SO\x00\x00\x00\x00\x00\x00\x00', result = 1 actual = 0
pass = b'pa\x00\x00\x00\x00\x00\x00\x00', result = 1 actual = 0
pass = b'SOS\x00\x00\x00\x00\x00\x00', result = 1 actual = 0
pass = b'pas\x00\x00\x00\x00\x00\x00', result = 1 actual = 0
pass = b'SOSN\x00\x00\x00\x00\x00', result = 1 actual = 0
pass = b'pass\x00\x00\x00\x00\x00', result = 1 actual = 0
pass = b'SOSNE\x00\x00\x00\x00', result = 1 actual = 0
pass = b'passw\x00\x00\x00\x00', result = 1 actual = 0
pass = b'SOSNEA\x00\x00\x00', result = 1 actual = 0
pass = b'passwo\x00\x00\x00', result = 1 actual = 0
pass = b'SOSNEAK\x00\x00', result = 1 actual = 0
pass = b'passwor\x00\x00', result = 1 actual = 0
pass = b'SOSNEAKY\x01', result = 1 actual = 0
pass = b'password\x01', result = 1 actual = 0
[b'S\x00\x00\x00\x00\x00\x00\x00\x00', b'p\x00\x00\x00\x00\x00\x00\x00\x00', b'SO\x00\x00\x00\x

In [19]:
p.loader.find_symbol("strcmp")

<Symbol "strcmp" in cle##externs at 0x500028>

In [29]:
strcmp = p.factory.callable(0x500028, concrete_only=True, prototype=prototype)

In [33]:
#bug?? don't think this should be happening to angr
strcmp("asdf","a")

<BV32 0x0>

In [35]:
strcmp("asdf","f")

<BV32 0xffffffff>

In [22]:
#Analysis between fauxware (auth_args) and fauxware-diff (auth_diff_args)

In [23]:
orig_diff_diff = []
for case in auth_args:
    if auth_diff(case[0]) != case[1]:
        print("pass = " + str(case[0]) + ", result = " + str(auth_diff(case[0])) +" actual = "+str(case[1]))
        orig_diff_diff.append(case[0])
print()
for case in auth_diff_args:
    if auth(case[0]) != case[1]:
        print("pass = " + str(case[0]) + ", result = " + str(auth(case[0])) +" actual = "+str(case[1]))
        orig_diff_diff.append(case[0])
print(orig_diff_diff)

pass = b'SOSNEAKY\x00', result = 0 actual = 1

pass = b'RNRMD@JX\x00', result = 0 actual = 1
pass = b'RNRMD@JX\x80', result = 0 actual = 1
[b'SOSNEAKY\x00', b'RNRMD@JX\x00', b'RNRMD@JX\x80']
